In [2]:
# Interactive Restaurant Density Mapping – Self‑Contained Script

# 1) Imports
import pandas as pd
import numpy as np
import folium
from folium.plugins import MarkerCluster
from IPython.display import IFrame

# 2) Load the datasets (ensure these CSVs are in your working directory)
data1 = pd.read_csv('data1.csv')
data2 = pd.read_csv('data2.csv')

# 3) Clean & preprocess data1
#   – Handle rate column (replace '-', 'NEW'; strip '/5'; convert to float; fill NaNs)
data1['rate'] = (data1['rate']
    .replace({'-': pd.NA, 'NEW': pd.NA})
    .str.replace('/5','', regex=False))
data1['rate'] = pd.to_numeric(data1['rate'], errors='coerce')
data1['rate'] = data1['rate'].fillna(data1['rate'].median())

#   – Handle cost column (remove commas; convert; fill NaNs)
data1['approx_costfor_two_people'] = (data1['approx_costfor_two_people']
    .astype(str)
    .str.replace(',','', regex=False))
data1['approx_costfor_two_people'] = pd.to_numeric(
    data1['approx_costfor_two_people'], errors='coerce')
data1['approx_costfor_two_people'] = data1['approx_costfor_two_people']\
    .fillna(data1['approx_costfor_two_people'].median())

#   – Fill missing text fields
data1['dish_liked'].fillna('Not Available', inplace=True)
data1['cuisines'].fillna('Other',         inplace=True)
data1['rest_type'].fillna('Unknown',      inplace=True)

#   – Votes to int
data1['votes'] = data1['votes'].fillna(data1['votes'].median()).astype(int)

#   – Binary encode
data1['online_order'] = data1['online_order'].map({'Yes':1,'No':0})
data1['book_table']  = data1['book_table'].map({'Yes':1,'No':0})

# 4) Merge with location data
merged_df = pd.merge(data1, data2, on='listed_incity', how='left')

# 5) Build the restaurant density map
density_map = folium.Map(location=[12.9716, 77.5946], zoom_start=12)
marker_cluster = MarkerCluster().add_to(density_map)

for _, row in merged_df.iterrows():
    lat, lon = row['Latitude'], row['Longitude']
    if pd.notnull(lat) and pd.notnull(lon):
        folium.Marker(
            location=[lat, lon],
            popup=(
                f"{row['listed_incity']}<br>"
                f"Rating: {row['rate']}/5<br>"
                f"Cost for Two: ₹{row['approx_costfor_two_people']}"
            ),
            tooltip=row['listed_incity']
        ).add_to(marker_cluster)

# 6) Save and display the map
density_map.save('restaurant_density.html')
IFrame('restaurant_density.html', width=900, height=600)
